In [1]:
import numpy as np
import pickle
import os, sys

### Problem 
- the irfs are loaded into memory twice. so everything else is really slow once they are loaded.

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath('./main_files'))

import numpy as np
import matplotlib.pyplot as plt
from MultinestClusterFit import MultinestClusterFit
from RebinningFunctions import spimodfit_binning_SE, log_binning_function_for_x_number_of_bins, no_rebinning #, rebin_data_exp_50
from PointingClusters import PointingClusters, save_clusters, load_clusters
from ModelSources import *
import pickle

In [12]:
sys.path.insert(0, os.path.abspath('./main_files'))
import astropy.io.fits as fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.time as at
from IntegralQuery import SearchQuery, IntegralQuery, Filter, Range

In [5]:
with open("error_test", "rb") as f:
    error_test = pickle.load(f)
error_test

(((array([[579., 494., 289., ..., 634., 623., 696.],
          [573., 452., 217., ..., 607., 659., 745.],
          [599., 459., 282., ..., 650., 626., 708.],
          ...,
          [496., 374., 218., ..., 577., 557., 662.],
          [623., 451., 256., ..., 663., 630., 686.],
          [611., 468., 297., ..., 634., 595., 698.]]),
   array([[507., 412., 192., ..., 654., 620., 747.],
          [587., 447., 256., ..., 644., 628., 712.],
          [605., 434., 243., ..., 601., 616., 743.],
          ...,
          [508., 390., 205., ..., 592., 564., 694.],
          [537., 363., 218., ..., 577., 623., 675.],
          [629., 445., 272., ..., 585., 643., 672.]])),
  (array([[527., 454., 222., ..., 615., 595., 712.],
          [578., 497., 278., ..., 590., 619., 694.],
          [548., 434., 239., ..., 647., 642., 692.],
          ...,
          [455., 369., 174., ..., 536., 573., 608.],
          [554., 429., 232., ..., 632., 637., 639.],
          [563., 466., 210., ..., 633., 624., 743

### Testing functions

In [ ]:
def extract_date_range(path):
    with fits.open(f"{path}/pointing.fits") as file:
        t = Table.read(file[1])
        t1 = at.Time(f'{t["TSTART"][0]+2451544.5}', format='jd')
        t1.format = "isot"
        t2 = at.Time(f'{t["TSTOP"][-1]+2451544.5}', format='jd')
        t2.format = "isot"
    return t1.value, t2.value

extract_date_range("./main_files/pure_simulation_tests/identical_repeats")

In [ ]:
def get_scw_ids(center_ra, center_dec, print_results=False):
    p = SkyCoord(center_ra, center_dec, frame="icrs", unit="deg")
    searchquerry = SearchQuery(position=p, radius=f"{10.} degree",)
    cat = IntegralQuery(searchquerry)

    scw_ids_all = None
    for path in ("./main_files/pure_simulation_tests/identical_repeats",):
        f = Filter(
            SCW_TYPE="POINTING",
            TIME={'min_val': extract_date_range(path)[0], 'max_val': extract_date_range(path)[1]}
        )
        if scw_ids_all:
            scw_ids_all = np.append(
                scw_ids_all,
                cat.apply_filter(f, return_coordinates=True, remove_duplicates=True),
                axis=0
            )
        else:
            scw_ids_all = cat.apply_filter(f, return_coordinates=True, remove_duplicates=True)
            

    print(scw_ids_all)
    scw_ids = []
    paths = []
    
    multiple_files = []
    no_files = []
    no_pyspi = []
    
    num_dets = 19
    eb = np.geomspace(18, 2000, 5)
    emod = np.geomspace(18, 2000, 5)
    
    for i, scw_id in enumerate(scw_ids_all):
        # Query may structure scws differently to our data, we only care about our data
        repeat = False
        for scw_id_2 in scw_ids_all[:i]:
            if scw_id[0][:8] == scw_id_2[0][:8]:
                repeat = True
                break
        if repeat:
            continue
        
        good = True
        found = False
        for path in self._orbit_paths:
            with fits.open(f"{path}/pointing.fits") as file:
                t = Table.read(file[1])
                index = np.argwhere(t["PTID_ISOC"]==scw_id[0][:8])
            
            if len(index) < 1:
                no_files.append(scw_id)
                continue
                                
            elif len(index) > 1:
                multiple_files.append(scw_id)
                good = False
                
            found = True
                                            
            pointing_info = t[index[-1][0]]
        
            t1 = at.Time(f'{pointing_info["TSTART"]+2451544.5}', format='jd').datetime
            time_start = datetime.strftime(t1,'%y%m%d %H%M%S')
                            
            with fits.open(f"{path}/dead_time.fits") as file2:
                t2 = Table.read(file2[1])
                
                time_elapsed = np.zeros(num_dets)
                
                for i in range(num_dets):
                    for j in index:
                        time_elapsed[i] += t2["LIVETIME"][j[0]*85 + i] # this is only true when combining fragmented scws
                            
            dets = get_live_dets(time=time_start, event_types=["single"])
                            
            if not np.amin(time_elapsed[dets]) > self._min_time_elapsed:
                good = False
                
            try: # investigate why this is necessary
                version = find_response_version(time_start)
                rsp = ResponseRMFGenerator.from_time(time_start, dets[0], eb, emod, rsp_bases[version])
            except:
                no_pyspi.append(scw_id)
                good = False
                
            if good:
                paths.append(path)
            
        if not found:
            good = False
            
        if good:
            scw_ids.append(scw_id)
                
    if print_results:
        print("Multiple Files:")
        print(multiple_files)
        print("No Files:")
        print(no_files)
        print("No PySpi:")
        print(no_pyspi)
        print("Good:")
        print(scw_ids)
        
    assert len(paths) == len(scw_ids), "At least one SCW is found in multiple paths"
    
    scw_ids = np.array(scw_ids)
    cw_paths = paths

get_scw_ids(10,-40)

$K \begin{cases} \left(\frac{x}{piv}\right)^{\alpha} \exp \left(-\frac{(2+\alpha) x}{x_{p}}\right) & x \leq (\alpha-\beta) \frac{x_{p}}{(\alpha+2)} \\ \left(\frac{x}{piv}\right)^{\beta} \exp (\beta-\alpha)\left[\frac{(\alpha-\beta) x_{p}}{piv(2+\alpha)}\right]^{\alpha-\beta} &x>(\alpha-\beta) \frac{x_{p}}{(\alpha+2)} \end{cases} $

Band function from Master-thesis

In [ ]:
os.environ['PYSPI_PACKAGE_DATA']

### reading pickle files

In [ ]:
# crab_fits.py sucht hier nach .fits dateien, aber nur pickle hier. vermutlich zu gross fuer github
x = np.load("main_files/SPI_data/0043/pointings.pickle", allow_pickle=True)
# das sind die pointings informationen schon weiter verarbeitet, mulitnestclusterfit braucht irgendwie auch noch die .fits dateien. mal schauen.

In [ ]:
x[0][0][0][:8] # das ist die pointing id soweit ich weiss. erste vier zahlen stehen moeglicherweisse fuer die revolution nr. oder id

In [ ]:
pointings = load_clusters(data_folder)

source_model = define_sources((
        (simulated_pl_0374, (200,)),
    ))

source_model.display()

In [ ]:
data_folder = "./main_files/pure_simulation_tests/identical_repeats"

# load  true params
with open(f"{data_folder}/source_params.pickle", "rb") as f:
    source_ra, source_dec, source_piv, source_Ks, source_indices = pickle.load(f)

with open(f"{data_folder}/pointings.pickle", "rb") as f:
    pointings_form_pickle = pickle.load(f)


# look at fitted source parameters for simulated test 1, run one (accidantly overwritten the pickle file for run 0).
temp_path = f"{data_folder}/1"

with open(f"{temp_path}/source_parameters.pickle", "rb") as f:
    val, cov = pickle.load(f) # prediction of the values K and index in this order

with open(f'{temp_path}/pointing_combinations', 'rb') as f:
    pointing_combinations = f.read()

print(f'predicted values: K={val[0]}, index={val[1]}')
print(f'true_values: K={source_Ks}, index={source_indices}')
print(f'pointing_combinations= {pointing_combinations}')


In [ ]:
p = load_clusters(data_folder)

In [15]:
with fits.open(f"/home/tguethle/Documents/sc_orbit_param.fits") as file:
        t = Table.read(file[1])
t.keys()

['OB_TIME',
 'REVOL',
 'REVOL_PHASE',
 'REVOL_FRAC',
 'DISTANCE',
 'XPOS',
 'YPOS',
 'ZPOS',
 'XVEL',
 'YVEL',
 'ZVEL',
 'RA_SCX',
 'DEC_SCX',
 'RA_SCZ',
 'DEC_SCZ',
 'POSANGLE']

In [ ]:
spiDataPath = '/home/tguethle/Documents/spi/Master_Thesis/main_files/SPI_data/test_data'


with fits.open(f"{spiDataPath}/pointing.fits") as file:
        t = Table.read(file[1])
t.keys()

In [ ]:
for i in range(len(t)):
    print(t[i]['PTID_SPI'])



In [ ]:
t['COUNTS']